# 4D Sphere Bayesian Optimisation Demo

## Overview

This notebook demonstrates **Bayesian Optimisation on a 4D sphere** ($S^3$) using geometric kernels. The 3-sphere is embedded in $\mathbb{R}^4$ and triangulated using Delaunay triangulation.

### Key Components

1. **Point Cloud & Triangulation**: 100 points sampled uniformly on $S^3$, then Delaunay triangulated
2. **Mesh Laplacian**: Custom eigendecomposition of the mesh Laplacian (cotangent formula)
3. **Geometric Kernel**: Matérn kernel adapted for manifold geometry using the Karhunen-Loève expansion
4. **Bayesian Optimisation**: Discrete BO on mesh vertices using Expected Improvement acquisition

### Contents

1. Package installation and imports
2. Mesh and kernel setup (loading pre-computed eigendecomposition)
3. Objective function definition
4. Bayesian Optimisation loop
5. Visualisation of BO progress
6. Kernel geometry-awareness demonstration
7. Convergence analysis

**Note**: Since the 4D sphere cannot be directly visualised, we project to 3D using the first three coordinates. This is for intuition only - the kernel operates in the full 4D space.

---

## 1. Package Installation and Imports

In [1]:
# Install required packages
!pip install pymanopt matplotlib ipympl kaleido plotly scipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 77.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 914.9/914.9 kB 4.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 54.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 49.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.0/35.0 MB 61.8 MB/s  0:00:006m0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 48.5 MB/s  0:00:006m0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 89.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 94.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 75.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25/25 [ipympl]24/25 [ipympl]ets]


In [2]:
# Core imports
import numpy as np
import random
from scipy.stats import norm
from pathlib import Path

# Geometric kernels
import geometric_kernels
from geometric_kernels.spaces import Mesh
from geometric_kernels.kernels import MaternKarhunenLoeveKernel
from geometric_kernels.spaces.eigenfunctions import EigenfunctionsFromEigenvectors

# Plotting
import matplotlib.pyplot as plt
import plotly
import plotly.io as pio
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Configure plotly for GitHub Codespaces (use 'notebook' renderer for inline display)
# Use 'notebook' for Jupyter environments like Codespaces, 'browser' for local
pio.renderers.default = 'notebook'

print("All imports successful!")

ModuleNotFoundError: No module named 'geometric_kernels'

---

## 2. Mesh and Kernel Setup

We load the pre-computed 4D sphere mesh and its Laplacian eigendecomposition. The eigenvalues and eigenvectors were computed using the n-dimensional cotangent formula.

In [ ]:
# Set up paths
MeshFolder_dir = Path.cwd()
print("Working directory:", MeshFolder_dir)

# Load the mesh (OBJ file for geometric_kernels compatibility)
Filename = "Delaunay 100point sphere.obj"
MESH = Mesh.load_mesh(str(MeshFolder_dir / Filename))
print(f"Mesh loaded with {MESH.num_vertices} vertices")

# Load pre-computed eigendecomposition
eigenvals = np.load(MeshFolder_dir / "4D_eigenvals.npy").reshape(-1, 1)
eigenvectors = np.load(MeshFolder_dir / "4D_eigenvecs.npy")  # Shape: (N, N) where N = num_vertices

# Load the original 4D points (for coordinate-based objective functions)
points_4D = np.load(MeshFolder_dir / "Delaunay_4D_points.npy")  # Shape: (N, 4)

print(f"Eigenvalues shape: {eigenvals.shape}")
print(f"Eigenvectors shape: {eigenvectors.shape}")
print(f"4D points shape: {points_4D.shape}")

In [ ]:
# Set up the domain (vertex indices)
num_verts = MESH.num_vertices
whole_domain = np.atleast_2d(np.arange(1, num_verts + 1)).T

# Create eigenfunctions from pre-computed eigenvectors
eigenfunctions = EigenfunctionsFromEigenvectors(eigenvectors=eigenvectors)

# Create the Matérn kernel with Karhunen-Loève expansion
# dimension=4 since we're on a 3-sphere (embedded in R^4)
kernel = MaternKarhunenLoeveKernel(
    space=MESH,
    eigenfunctions=eigenfunctions,
    eigenvalues_laplacian=eigenvals,
    num_levels=eigenvectors.shape[0],
    dimension=4,  # 4D embedding dimension
    normalize=True
)

# Kernel hyperparameters
LENGTH_SCALE = 0.3  # Controls correlation decay with geodesic distance
NU = 0.4  # Smoothness parameter

# Initialize kernel parameters
params = kernel.init_params()
params["lengthscale"] = np.array([LENGTH_SCALE])
params["nu"] = np.array([NU])

print(f"Kernel configured with lengthscale={LENGTH_SCALE}, nu={NU}")
print(f"Domain size: {num_verts} vertices")

---

## 3. Objective Function

We define a simple objective function based on the first 4D coordinate. This gives us a function with a clear minimum on the sphere.

In [ ]:
def objective_function(x):
    """
    Objective function: returns the first coordinate of the 4D point.
    
    Args:
        x: Node index (1-indexed) or array of indices
    
    Returns:
        The first coordinate value(s)
    """
    if isinstance(x, np.ndarray):
        idx = np.int64(x.flatten()) - 1  # Convert to 0-indexed
        return points_4D[idx, 0].flatten()[0] if len(idx) == 1 else points_4D[idx, 0]
    else:
        return points_4D[x - 1, 0]

# Vectorised version for batch evaluation
objective_vectorized = np.vectorize(objective_function)

# Compute objective values for all vertices
objective_vals = objective_vectorized(np.arange(1, num_verts + 1))

print(f"Objective value range: [{objective_vals.min():.4f}, {objective_vals.max():.4f}]")
print(f"True minimum value: {objective_vals.min():.4f} at index {np.argmin(objective_vals) + 1}")

---

## 4. Bayesian Optimisation Functions

We implement the Expected Improvement acquisition function and the main BO loop.

In [ ]:
# Exploration-exploitation trade-off: higher xi encourages more exploration
XI_DEFAULT = 0.9

def expected_improvement(mu, sigma, f_best, xi=XI_DEFAULT):
    """
    Calculate Expected Improvement (EI) acquisition function.
    
    Args:
        mu: Posterior mean vector
        sigma: Posterior standard deviation vector
        f_best: Best observed function value (we minimise)
        xi: Exploration-exploitation trade-off parameter (default: 0.9 for exploration)
    
    Returns:
        EI values for each point
    """
    mu = mu.reshape(-1, 1)
    sigma = sigma.reshape(-1, 1)
    
    with np.errstate(divide='ignore', invalid='ignore'):
        Z = (f_best - mu - xi) / sigma
        ei = (f_best - mu - xi) * norm.cdf(Z) + sigma * norm.pdf(Z)
        ei = np.where(sigma > 1e-10, ei, 0.0)
    
    return ei

In [ ]:
def BO_loop(num_iterations, x_obs=None, objective_func=None):
    """
    Run Bayesian Optimisation loop.
    
    Args:
        num_iterations: Number of BO iterations
        x_obs: Initial observed points (1-indexed vertex indices)
        objective_func: Objective function to optimise
    
    Returns:
        Tuple of (posterior_mean, posterior_var, EI, observed_X, observed_Y)
    """
    if objective_func is None:
        objective_func = objective_function
    if x_obs is None:
        x_obs = np.array([[np.random.randint(1, num_verts + 1)]])
    
    # Evaluate objective at initial points
    y_observed = np.atleast_2d(
        np.apply_along_axis(objective_func, 1, x_obs)
    ).reshape(-1, 1)
    
    # Prior covariance matrix
    K_XX_prior = kernel.K(params, whole_domain - 1, whole_domain - 1)
    mu_prior_vector = np.zeros((num_verts, 1))
    
    for i in range(num_iterations):
        # GP posterior computation
        m_vector = mu_prior_vector[x_obs.flatten() - 1]
        K_xX = kernel.K(params, x_obs - 1, whole_domain - 1)
        K_xx = kernel.K(params, x_obs - 1, x_obs - 1)
        K_Xx = K_xX.T
        
        # Add regularisation for numerical stability (jitter term)
        # 1e-6 is a standard choice to prevent singular matrices while preserving accuracy
        JITTER = 1e-6
        K_xx_stable = K_xx + np.eye(K_xx.shape[0]) * JITTER
        C_inv = np.linalg.pinv(K_xx_stable)
        
        # Posterior mean and variance
        mew_vec = mu_prior_vector + K_Xx @ C_inv @ (y_observed - m_vector)
        Current_K_matrix = K_XX_prior - K_Xx @ C_inv @ K_xX
        Sigma_vec = np.diag(Current_K_matrix).copy().reshape(-1, 1)
        Sigma_vec[Sigma_vec < 0] = 0  # Ensure non-negative variance
        
        # Compute acquisition function
        EI_vec = expected_improvement(mew_vec, np.sqrt(Sigma_vec), np.min(y_observed))
        
        # Select next point (greedy)
        next_point = np.argmax(EI_vec) + 1
        next_point = np.atleast_2d(next_point)
        
        # Evaluate and update
        y_next = np.atleast_2d(objective_func(next_point))
        x_obs = np.vstack((x_obs, next_point))
        y_observed = np.vstack((y_observed, y_next))
    
    return mew_vec, Sigma_vec, EI_vec, x_obs, y_observed

---

## 5. Run Bayesian Optimisation

We run BO with different iteration counts to observe how the posterior evolves.

In [ ]:
# Set random seed for reproducibility
np.random.seed(42)

# Random initial point
initial_point = np.array([[np.random.randint(1, num_verts + 1)]])
print(f"Initial point: vertex {initial_point[0,0]}")
print(f"Initial objective value: {objective_vals[initial_point[0,0]-1]:.4f}")

# Run BO with different iteration counts
mu_1, sigma_1, ei_1, X_1, Y_1 = BO_loop(1, x_obs=initial_point.copy())
mu_2, sigma_2, ei_2, X_2, Y_2 = BO_loop(5, x_obs=initial_point.copy())
mu_3, sigma_3, ei_3, X_3, Y_3 = BO_loop(10, x_obs=initial_point.copy())

print("\n--- Results Summary ---")
print(f"After 1 iteration:  Best observed = {Y_1.min():.4f}")
print(f"After 5 iterations: Best observed = {Y_2.min():.4f}")
print(f"After 10 iterations: Best observed = {Y_3.min():.4f}")
print(f"\nTrue minimum: {objective_vals.min():.4f} at vertex {np.argmin(objective_vals) + 1}")

---

## 6. Visualisation Helper Functions

Since we're visualising a 4D sphere, we project to 3D using the first three coordinates.

In [ ]:
def update_figure(fig):
    """Apply clean styling to a plotly figure."""
    fig.update_layout(scene_aspectmode="cube")
    fig.update_scenes(xaxis_visible=False, yaxis_visible=False, zaxis_visible=False)
    fig.update_layout(margin=dict(l=0, r=0, t=40, b=0))
    fig.update_layout(plot_bgcolor="rgba(0,0,0,0)", paper_bgcolor="rgba(0,0,0,0)")
    fig.update_layout(
        scene=dict(
            xaxis=dict(showbackground=False, showticklabels=False, visible=False),
            yaxis=dict(showbackground=False, showticklabels=False, visible=False),
            zaxis=dict(showbackground=False, showticklabels=False, visible=False),
        )
    )
    return fig

def create_mesh_trace(values, name, colorscale='Viridis', showscale=True):
    """
    Create a 3D scatter trace for mesh vertices colored by values.
    Projects 4D points to 3D using first three coordinates.
    """
    values_flat = values.ravel()
    
    # Use first 3 coordinates for 3D projection
    trace = go.Scatter3d(
        x=points_4D[:, 0],
        y=points_4D[:, 1],
        z=points_4D[:, 2],
        mode='markers',
        name=name,
        marker=dict(
            size=6,
            color=values_flat,
            colorscale=colorscale,
            showscale=showscale,
            colorbar=dict(title=name, x=0.9) if showscale else None
        ),
        customdata=np.column_stack([values_flat, np.arange(1, num_verts + 1), points_4D[:, 3]]),
        hovertemplate=(
            f"{name}: %{{customdata[0]:.4f}}<br>" +
            "Vertex: %{customdata[1]}<br>" +
            "4th coord: %{customdata[2]:.4f}<br>" +
            "<extra></extra>"
        )
    )
    return trace

def create_points_trace(X_indices, name, color):
    """Create a scatter trace for observed points."""
    coords = points_4D[X_indices.flatten() - 1]
    trace = go.Scatter3d(
        x=coords[:, 0],
        y=coords[:, 1],
        z=coords[:, 2],
        mode='markers',
        name=name,
        marker=dict(size=10, color=color, symbol='diamond')
    )
    return trace

---

## 7. Visualise BO Progress

Compare the posterior mean at different iteration counts with the true objective.

In [ ]:
# Create figure comparing posteriors
fig = go.Figure()

# Add traces for different iterations
fig.add_trace(create_mesh_trace(mu_1, 'Posterior (1 iter)', 'Blues', showscale=False))
fig.add_trace(create_mesh_trace(mu_2, 'Posterior (5 iter)', 'Greens', showscale=False))
fig.add_trace(create_mesh_trace(mu_3, 'Posterior (10 iter)', 'Oranges', showscale=False))
fig.add_trace(create_mesh_trace(objective_vals, 'True Objective', 'hot', showscale=True))

# Add observed points
fig.add_trace(create_points_trace(X_2, 'Sampled (5 iter)', 'cyan'))
fig.add_trace(create_points_trace(X_3, 'Sampled (10 iter)', 'purple'))

# Style and show
fig = update_figure(fig)
fig.update_layout(
    title='Bayesian Optimisation on 4D Sphere (projected to 3D)',
    legend=dict(x=0.02, y=0.98)
)
fig.show()

In [ ]:
# Print summary of sampled values
print("\n--- Sampled Values (sorted, ascending) ---")
print(f"After 5 iterations: {sorted(Y_2.flatten())}")
print(f"After 10 iterations: {sorted(Y_3.flatten())}")
print(f"\nTrue best value: {objective_vals.min():.4f}")
print(f"Initial point value: {objective_vals[initial_point[0,0]-1]:.4f}")

---

## 8. Kernel Geometry-Awareness Demonstration

Visualise how the kernel spreads covariance from a single source point across the mesh. Points with high kernel values are more correlated with the source point.

In [ ]:
def visualize_kernel_influence(source_point):
    """
    Visualise kernel influence from a single source point.
    
    Args:
        source_point: 0-indexed source vertex
    
    Returns:
        Plotly figure
    """
    source_idx = np.atleast_2d([source_point])
    all_points = np.atleast_2d(np.arange(num_verts)).T
    
    # Compute kernel values from source to all points
    K_influence = kernel.K(params, source_idx, all_points).flatten()
    
    # Create figure
    fig = go.Figure()
    
    # Add influence trace
    fig.add_trace(create_mesh_trace(K_influence, 'Kernel Value', 'Viridis', showscale=True))
    
    # Mark source point
    source_coord = points_4D[source_point]
    fig.add_trace(go.Scatter3d(
        x=[source_coord[0]], 
        y=[source_coord[1]], 
        z=[source_coord[2]],
        mode='markers',
        marker=dict(size=15, color='red', symbol='diamond'),
        name='Source Point'
    ))
    
    fig = update_figure(fig)
    fig.update_layout(title=f"Kernel Influence from Vertex {source_point + 1}")
    
    return fig, K_influence

# Visualise kernel influence from a specific point
source_vertex = 3  # 0-indexed
influence_fig, kernel_values = visualize_kernel_influence(source_vertex)
influence_fig.show()

print(f"\nKernel influence statistics:")
print(f"  Min: {kernel_values.min():.4f}")
print(f"  Max: {kernel_values.max():.4f} (at source)")
print(f"  Mean: {kernel_values.mean():.4f}")

---

## 9. BO Convergence Analysis

Plot how the best observed value improves over iterations.

In [ ]:
# Run longer BO experiment for convergence analysis
np.random.seed(123)
n_iterations = 20
initial_point_conv = np.array([[np.random.randint(1, num_verts + 1)]])

# Collect convergence data
best_values = []
x_obs_conv = initial_point_conv.copy()
y_obs_conv = np.atleast_2d(objective_function(x_obs_conv)).reshape(-1, 1)
best_values.append(y_obs_conv.min())

for i in range(n_iterations):
    mu_temp, sigma_temp, ei_temp, x_obs_conv, y_obs_conv = BO_loop(
        1, x_obs=x_obs_conv, objective_func=objective_function
    )
    best_values.append(y_obs_conv.min())

# Plot convergence
fig_conv, ax = plt.subplots(figsize=(10, 6))
ax.plot(range(len(best_values)), best_values, 'b-o', linewidth=2, markersize=8, label='Best Observed')
ax.axhline(y=objective_vals.min(), color='r', linestyle='--', linewidth=2, label=f'True Minimum ({objective_vals.min():.4f})')
ax.set_xlabel('Iteration', fontsize=12)
ax.set_ylabel('Best Objective Value', fontsize=12)
ax.set_title('BO Convergence on 4D Sphere', fontsize=14)
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('bo_convergence_plot.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\nConvergence Summary:")
print(f"  Initial value: {best_values[0]:.4f}")
print(f"  Final value:   {best_values[-1]:.4f}")
print(f"  True minimum:  {objective_vals.min():.4f}")
print(f"  Gap to optimal: {best_values[-1] - objective_vals.min():.4f}")

---

## 10. Additional Analysis: Kernel vs Geodesic Distance

Examine the relationship between kernel values and approximate geodesic distance.

In [ ]:
# Compute pairwise kernel matrix
K_full = kernel.K(params, whole_domain - 1, whole_domain - 1)

# Compute Euclidean distances (approximation of geodesic on sphere)
from scipy.spatial.distance import pdist, squareform
euclidean_dists = squareform(pdist(points_4D, 'euclidean'))

# For a sphere, geodesic distance ≈ arcsin(chord_distance/2) * 2 * radius
# For unit sphere: geodesic ≈ 2 * arcsin(euclidean/2)
geodesic_approx = 2 * np.arcsin(np.clip(euclidean_dists / 2, 0, 1))

# Scatter plot: kernel value vs geodesic distance
fig, ax = plt.subplots(figsize=(10, 6))

# Extract upper triangle (excluding diagonal)
upper_idx = np.triu_indices(num_verts, k=1)
geodesic_flat = geodesic_approx[upper_idx]
kernel_flat = K_full[upper_idx]

ax.scatter(geodesic_flat, kernel_flat, s=3, alpha=0.5, c='blue')
ax.set_xlabel('Approximate Geodesic Distance', fontsize=12)
ax.set_ylabel('Kernel Value', fontsize=12)
ax.set_title('Kernel Decay with Geodesic Distance on 4D Sphere', fontsize=14)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nThe kernel respects geodesic geometry:")
print("- High kernel values for nearby points (small geodesic distance)")
print("- Low kernel values for distant points (large geodesic distance)")

---

## Summary

This demo showed:

1. **Custom Eigendecomposition**: Loading pre-computed mesh Laplacian eigenvectors/eigenvalues for a 4D sphere
2. **Geometric Kernel**: Using `MaternKarhunenLoeveKernel` with dimension=4 to properly scale the kernel
3. **Bayesian Optimisation**: Discrete BO on mesh vertices using Expected Improvement
4. **Visualisation**: 3D projection of the 4D sphere for intuition
5. **Geometry-Awareness**: The kernel correctly captures geodesic relationships on the sphere

### Key Insights

- The kernel exploits the manifold structure to spread information based on geodesic distance
- BO converges to near-optimal solutions within a few iterations
- The 3D projection is for visualisation only; the kernel operates in the full 4D space